In [ ]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes = True)
#from sqlalchemy import create_engine
from pylab import rcParams
from matplotlib import pyplot
from datetime import datetime, timedelta


import math
import keras
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout
#from keras.layers import *
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error
# from keras.callbacks import EarlyStopping
# from sklearn import metrics

In [ ]:
df = pd.read_csv('/Users/tarondavtyan/Downloads/for_prediction_30_min.csv', index_col = 0)

In [ ]:
# df['Date'] = pd.to_datetime(df['Time'], errors='coerce')

In [ ]:
df.set_index('Time', inplace=True)

In [ ]:
df.head()

In [ ]:
# df['day-1'] = df['Number of Trips'].shift(periods = -1)

In [ ]:
df['day-1'].ffill(inplace = True )

In [ ]:
df['week-1'] = df['Number of Trips'].shift(periods = -336)

In [ ]:
df['week-1'].ffill(inplace = True )

In [ ]:
ax = sns.catplot(x='month',kind='count',data=df,orient="h")
ax.fig.autofmt_xdate()

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df = df[np.abs(df["Count"]-df["Count"].mean())<=(3*df["Count"].std())]

In [ ]:
df.shape

In [ ]:
df.rename(columns = {'Count': 'Number of Trips','Tempreture' : 'Temperature' }, inplace = True)

In [ ]:
#corr = np.corrcoef(np.random.randn(10, 200))
mask = np.zeros_like(df.corr())
mask[np.triu_indices_from(mask)] = True

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(18,10))
ax = sns.heatmap(df.corr(),
           cmap='coolwarm',
           annot = True,
           annot_kws={"size": 20}, 
           mask = mask,
            fmt='.2f'
          )
ax.set_xticklabels(df.columns,size = 25, fontname="Arial", rotation = 90)
ax.set_yticklabels(df.columns, size = 25, fontname="Arial")

fig = ax.get_figure()
plt.savefig('/Users/tarondavtyan/Desktop/Master_thesis_plots/11. corr.pdf',bbox_inches='tight')

In [ ]:
plt.figure(figsize=(18,10))
sns.heatmap(df.corr(),
           cmap='coolwarm',
           annot = True,
           annot_kws={"size": 12}
           #mask = mask
          );

# Machine Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

# RANDOM_SEED = 42

# np.random.seed(RANDOM_SEED)
# tf.random.set_random_seed(1)

In [ ]:
def gradient_importance(seq, model):
    seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)
    with tf.GradientTape() as tape:
        predictions = model(seq)
    grads = tape.gradient(predictions, seq)
    grads = tf.reduce_mean(grads, axis=1).numpy()[0]
    
    return grads

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
df_fold1 = int(len(df) * 0.1)
df_fold2 = int(len(df) * 0.2)
df_fold3 = int(len(df) * 0.3)
df_fold4 = int(len(df) * 0.4)
df_fold5 = int(len(df) * 0.5)
df_fold6 = int(len(df) * 0.6)
df_fold7 = int(len(df) * 0.7)
df_fold8 = int(len(df) * 0.8)
df_fold9 = int(len(df) * 0.9)
df_fold10 = int(len(df) * 1)

In [ ]:
# train_size = int(len(df) * 0.9)
# test_size = len(df) - train_size
#train, test = pd.concat([df.iloc[0:df_fold1], df.iloc[df_fold2:len(df)]]), df.iloc[df_fold1:df_fold2]
#train, test = df.iloc[df_fold1:df_fold10], df.iloc[0:df_fold1]
train, test = df.iloc[0:df_fold9], df.iloc[df_fold9:df_fold10]
print(len(train), len(test))


In [ ]:
test.head()

In [ ]:
test_for_plot=test.reset_index()

In [ ]:
test_for_plot.head()

In [ ]:
test_for_plot['Time'] =  pd.to_datetime(test_for_plot['Time'])

In [ ]:
test_for_plot['dates'] = test_for_plot['Time'].dt.date

In [ ]:
test_for_plot = test_for_plot[10:851]

In [ ]:
test_for_plot.info()

In [ ]:
df.columns

In [ ]:
time_steps = 10

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train['Number of Trips'], time_steps)
X_test, y_test = create_dataset(test, test['Number of Trips'], time_steps)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
model = keras.Sequential()
model.add(
  keras.layers.Bidirectional(
    keras.layers.LSTM(
      units=128, 
      input_shape=(X_train.shape[1], X_train.shape[2])
    )
  )
)
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_loss', mode='auto')
history = model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.1,
    callbacks=[es],
    shuffle=False
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

In [ ]:
import matplotlib.dates as mdates
monthyearFmt = mdates.DateFormatter('%d-%m')

In [ ]:
plt.figure(figsize=(18,10))
plt.plot( y_test, marker='.', label="Observation")
plt.plot(y_pred, 'r', label="Prediction")
plt.ylabel('Number of Trips', fontsize = 35)
plt.xlabel('Time Step', fontsize = 35)
#plt.xticks(np.arange(0,1000, 300), fontsize= 25)
plt.yticks(fontsize= 25)
plt.legend(prop={'size':25})


#plt.show()
plt.savefig('/Users/tarondavtyan/Desktop/Master_thesis_plots/10. prediction.pdf',bbox_inches='tight')

In [ ]:
idx = pd.date_range('2017-09-13', '2017-10-01')
s = pd.Series(np.random.randn(len(idx)), index=idx)

In [ ]:
df_ytest = pd.DataFrame({'Column1': y_test[[:,0],:]})

In [ ]:
from pandas import DataFrame

df_ytest = DataFrame (y_test,columns=['Column_Name'])

In [ ]:
df_ytest.head()

In [ ]:
result_test.head()

In [ ]:
test_for_plot_new = test_for_plot['Time']

In [ ]:
df_new = test_for_plot['Time'].reset_index()

In [ ]:
result_test = pd.concat([df_ytest,df_new], axis = 1 )

In [ ]:
result_test.head()

In [ ]:
result_test = result_test[['Column_Name', 'Time']]

In [ ]:
result_test.rename(columns = {'Column_Name': 'values'}, inplace = True)

In [ ]:
df_ypred = DataFrame (y_pred,columns=['values'])

In [ ]:
result_pred = pd.concat([df_ypred,df_new], axis = 1 )

In [ ]:
result_pred = result_pred[['values', 'Time']]

In [ ]:
result_pred.head()

In [ ]:
result_test.head()

In [ ]:
import matplotlib.dates as mdates

In [ ]:
ax1 = sns.lineplot(x='Time', y='values', data=result_test,  marker = 'o', label = 'Observation')

ax1 = sns.lineplot(x='Time', y='values', data=result_pred, color = 'red', label = 'Prediction')

date_form = mdates.DateFormatter("%d/%m/%Y \n%a")
ax1.xaxis.set_major_formatter(date_form)
ax1.xaxis.set_major_locator(mdates.DayLocator(bymonthday=range(10,32), interval = 5))
#ax1.set_xticklabels(ax1.get_yticks(), size = 15)

plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
ax1.set_xlabel('', fontsize = 30)  
ax1.set_ylabel('Number of Trips', fontsize = 30)  


# ax1.set_xticks(range(len(result_test))) # <--- set the ticks first
# ax1.set_xticklabels(['2011','2012','2013','2014','2015','2016','2017','2018'])

plt.legend(loc=1, prop={'size': 25})

plt.savefig('/Users/tarondavtyan/Desktop/Master_thesis_plots/10. prediction.pdf',bbox_inches='tight')

In [ ]:
#with 0.1 validation
print('MAE:       %f' % metrics.mean_absolute_error(y_test, y_pred))
print('MSE:       %f' % metrics.mean_squared_error(y_test, y_pred))
print('RMSE:      %f' % np.sqrt(metrics.mean_squared_error(y_test,y_pred)))



print('R-squared: %f' % metrics.explained_variance_score(y_test,y_pred))

In [ ]:
id_ = 10

In [ ]:
df.Count.mean()

In [ ]:
df.columns

In [ ]:
columns = ['Number of Trips', 'Tempreture', 'Precipitation', 'Hour', 'month', 'season_2',
       'season_3', 'season_4', 'weekday_0', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'holiday']

In [ ]:
grad_imp = gradient_importance(X_test[id_],model)

plt.figure(figsize=(15,10))
plt.bar(range(len(grad_imp)), grad_imp)
plt.xticks(range(len(columns)), columns, rotation=90)
plt.ylabel('gradients'); #plt.title(pd.to_datetime(test_date[sequence_length+id_]))
np.set_printoptions(False)

# Version 2 (with MinMax scaler)

In [ ]:
dataset = df.Count.values #numpy.ndarray
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
train_size = int(len(dataset) * 0.90)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)


In [ ]:
# reshape into X=t and Y=t+1
look_back = 15
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

In [ ]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, Y_train, epochs=40, batch_size=70, #validation_data=(X_test, Y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False)

# Training Phase
model.summary()

In [ ]:
# make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
# invert predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])

print('Train Mean Absolute Error:', mean_absolute_error(Y_train[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_train[0], train_predict[:,0])))
print('Test Mean Absolute Error:', mean_absolute_error(Y_test[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test[0], test_predict[:,0])))

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Bike Count')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
aa=[x for x in range(200)]
plt.figure(figsize=(8,4))
plt.plot(aa, Y_test[0][:200], marker='.', label="actual")
plt.plot(aa, test_predict[:,0][:200], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();